#### 1. Import pandas library

In [28]:
import pandas as pd
import numpy as np

#### 2. Import BigQuery library


In [29]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/JCasado/Desktop/IRONHACK/ironhack_service_account_big_query.json"
from google.cloud import bigquery

#### 3. Create a connection to BigQuery

In [30]:
client = bigquery.Client()
#nos deja hacer la conexion

#### 4. Import data from the users table from the stackoverflow public database for United Kingdom only

In [31]:
sql='''
SELECT 
*
FROM
    `bigquery-public-data.stackoverflow.users`
WHERE
    location = "United Kingdom"
'''


In [32]:
query_job = client.query(query=sql)
df = query_job.to_dataframe()

#### 5. Rename id column to user_id

In [33]:
data1 = df.rename(columns = {'id':'user_id'})

#### 6. Import from the stackoverflow_posts table all items which have up to 20 view counts and only select 'id', 'owner_user_id', 'score', 'view_count', 'comment_count', 'favorite_count'

In [34]:
sql2='''
SELECT
    id,
    owner_user_id,
    score,
    view_count,
    comment_count,
    favorite_count
FROM
    `bigquery-public-data.stackoverflow.stackoverflow_posts`
WHERE
    view_count < 20
;
'''

In [35]:
query_job = client.query(query = sql2)
df2 = query_job.to_dataframe()

#### 7. Rename id column to post_id and owner_user_id to user_id

In [36]:
data2 = df2.rename(columns = {'id':'post_id',
                     'owner_user_id':'user_id'})
data2.head()

,post_id,user_id,score,view_count,comment_count,favorite_count
0,37516574,NaN,0,7,0,NaN
1,37319831,NaN,1,18,0,NaN
2,31140287,NaN,1,16,3,NaN
3,36546336,NaN,0,8,0,NaN
4,37135854,NaN,0,12,0,NaN


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [64]:
users_columns = data1[["user_id","reputation","views","up_votes","down_votes"]]
users_columns.head()

,user_id,reputation,views,up_votes,down_votes
0,9493,95743,2394,579,7
1,125118,46,27,0,0
2,170346,2754,181,93,5
3,414284,782,74,41,6
4,656965,157,41,16,0


In [65]:
posts_columns = data2[["post_id","score","user_id","view_count","comment_count"]]
posts_columns.head()

,post_id,score,user_id,view_count,comment_count
0,37516574,0,NaN,7,0
1,37319831,1,NaN,18,0
2,31140287,1,NaN,16,3
3,36546336,0,NaN,8,0
4,37135854,0,NaN,12,0


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [66]:
'''
DataFrame.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, 
suffixes=('_x', '_y'), copy=True, indicator=False, validate=None) → 'DataFrame

'''

"\nDataFrame.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, \nsuffixes=('_x', '_y'), copy=True, indicator=False, validate=None) → 'DataFrame\n\n"

In [67]:
users_columns = users_columns.merge(posts_columns, left_on = 'user_id', right_on = 'user_id')
#lefton righton parametro comun interseccion

In [68]:
users_columns.head()

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,1499072,2073,340,1964,13,34118493,0,13,0
1,1499072,2073,340,1964,13,35837533,1,9,0
2,1499072,2073,340,1964,13,36560530,0,18,6
3,1607639,63,23,2,0,36080095,0,14,0
4,218208,2040,464,325,19,35920077,0,18,3


#### 9. How many missing values do you have in your merged dataframe? 

In [69]:
users_columns.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 931 entries, 0 to 930
Data columns (total 9 columns):
user_id          931 non-null int64
reputation       931 non-null int64
views            931 non-null int64
up_votes         931 non-null int64
down_votes       931 non-null int64
post_id          931 non-null int64
score            931 non-null int64
view_count       931 non-null int64
comment_count    931 non-null int64
dtypes: int64(9)
memory usage: 72.7 KB


In [71]:
users_columns.isnull().sum()

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [72]:
users_columns = users_columns.drop("post_id", axis=1)

In [73]:
low_variance = []


for col in users_columns._get_numeric_data():
    minimum = min(users_columns[col])
    ninety_perc = np.percentile(users_columns[col], 90)
    
    if ninety_perc == minimum:
        low_variance.append(col)

In [74]:
print(low_variance)

[]


In [75]:
users_columns[low_variance].head()

""
0
1
2
3
4


In [ ]:
#intento hacer lo mismo que en code along pero no me hace ningun append a la tabla

In [77]:
stats = users_columns.describe().transpose()
stats["IQR"] = stats["75%"] - stats["25%"]
stats

,count,mean,std,min,25%,50%,75%,max,IQR
user_id,931.0,2.223184e+06,1.823178e+06,274.0,630780.0,1807486.0,3480133.0,6449207.0,2849353.0
reputation,931.0,7.745998e+03,2.510228e+04,1.0,243.0,1127.0,5299.0,279695.0,5056.0
views,931.0,6.434189e+02,1.713170e+03,0.0,39.5,142.0,500.0,27257.0,460.5
up_votes,931.0,4.007229e+02,7.266345e+02,0.0,17.5,106.0,405.0,7845.0,387.5
down_votes,931.0,4.821697e+01,1.719530e+02,0.0,0.0,3.0,21.0,2549.0,21.0
score,931.0,1.245972e-01,5.988759e-01,-7.0,0.0,0.0,0.0,4.0,0.0
view_count,931.0,1.488507e+01,3.515664e+00,3.0,13.0,16.0,18.0,19.0,5.0
comment_count,931.0,8.829216e-01,1.600483e+00,0.0,0.0,0.0,1.0,11.0,1.0


In [80]:
outliers = pd.DataFrame(columns=users_columns.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = data[(data[col] < lower) |
                   (data[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)

outliers

KeyError: 'score'

In [ ]:
#me da error de keyError score, no se porque